# Домашнее задание 5. Линейные модели

In [ ]:
import random as pr
import numpy as np
import pandas as pd
import matplotlib.pylab as pl
import sklearn.cross_validation as cv
import sklearn.metrics as sm
from scipy.special import expit as sigmoid

# Plotting config
%pylab inline

Зачитываем результат 4 домашки

In [ ]:
data = np.load("out_4.dat.npz")
users = data["users"]
X = data["data"].reshape(1,)[0]

Зачитываем категории пользователей

In [ ]:
TRAINING_SET_URL = "twitter_train.csv"
df_users = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["twitter_id", "is_1", "is_2", "is_3"], dtype={"twitter_id": str, "is_1": int, 'is_2': int, "is_3": int})
df_users.set_index("twitter_id", inplace=True)

Формируем целевую переменную: Делаем join списка пользователей из ДЗ4 с обучающей выборкой.

In [ ]:
def f(x):
    if x[0] == 1:
        return 1
    if x[1] == 1:
        return 2
    if x[2] == 1:
        return 3

Y = df_users[['is_1', 'is_2', 'is_3']].apply(f, axis=1).values
print "Resulting training set: (%dx%d) feature matrix, %d target vector" % (X.shape[0], X.shape[1], Y.shape[0])

Чтобы исследовать, как ведут себя признаки, построим распределение количества ненулевых признаков у пользователей, чтобы убедиться, что он удовлетворяет закону Ципфа. Для этого построим гистограмму в логарифмических осях. [Подсказка](http://anokhin.github.io/img/sf1.png)

In [ ]:
def draw_log_hist(x):
    """Draw tokens histogram in log scales"""
    
    # Your code here
    features_count= x.getnnz(axis=0)
    result = np.sort(features_count)
    result = result[::-1]
    x = np.arange(len(result))
    fig = plt.figure(figsize=(12, 9))
    plt.loglog(x, result, marker='o', linestyle='None', markersize=3)
    plt.show()
    return features_count

features_counts = draw_log_hist(X)

Проведем отбор признаков. В самом простом случае просто удаляем признаки, имеющие ненулевое значение у менее, чем 100 пользователей.

In [ ]:
X1 = X.tocsc()[:, features_counts > 100].toarray()

Вариант задания генерируется на основании вашего ника в техносфере.

In [ ]:
USER_NAME = "n.teplyakova"
OPTIMIZATION_ALGORITHMS = ["stochastic gradient descent", "Newton method"]
REGULARIZATIONS = ["L1", "L2"]

print "My homework 5 algorithm is: Logistic regression with %s regularization optimized by %s" % (
    REGULARIZATIONS[hash(USER_NAME) % 2],
    OPTIMIZATION_ALGORITHMS[hash(USER_NAME[::-1]) % 2]
)

Реализуем выбранный алгоритм

In [ ]:
class LogisticRegression():
    def __init__(self, C=1.0, eps=0.001, max_iter=20000):
        #regularization parameter
        self.C = C
        self.eps = eps
        self.max_iter = max_iter
        
    
    def gradient_descent(self, w0, X, y):
        w = w0
        k = 0
        eta = 1
        eps = self.eps
        
        while True:
            reg = self.C * np.sign(w)
            reg[0, 0] = 0
            gradient = (X.transpose() * (sigmoid(X * w) - y) + reg) / X.shape[0]
            
            w = w - eta * gradient
            k += 1
            
            if k > self.max_iter or np.max(np.abs(gradient)) < eps:
                break
        
        print k
        w = np.array(w.flatten())
        return w[0]
    
    
    def fit_two_classes(self, X, y):
        w0 = np.matrix(np.zeros_like(X[0])).transpose()
        return self.gradient_descent(w0, X, y)
        
    
    def fit(self, X_in, Y):
        X = np.ones((X_in.shape[0], X_in.shape[1] + 1))
        X[:, 1:] = X_in
        X = np.matrix(X)
        
        self.classes = np.unique(Y)
        self.coef = []
        for cl in self.classes:
            y = np.matrix(np.vectorize(lambda x: 1 if x == cl else 0)(Y)).transpose()
            self.coef.append(self.fit_two_classes(X, y))
        return self
    
    
    def predict_proba(self, X_in):
        X = np.ones((X_in.shape[0], X_in.shape[1] + 1))
        X[:, 1:] = X_in
        
        probabilities = []
        for x in X:
            pr = [] 
            for coef in self.coef:
                pr.append(sigmoid(np.dot(x, coef)))
            pr = np.array(pr)
            probabilities.append(pr/np.sum(pr))
        
        return np.array(probabilities)

In [ ]:
from sklearn.datasets import load_iris
from matplotlib import cm
iris = load_iris()
X_iris = iris.data[:100, :2]
y_iris = iris.target[:100]

fig = plt.figure(figsize=(8, 8))
plt.scatter(X_iris[:, 0], X_iris[:, 1], c=y_iris, cmap=cm.Set1, s=40)
plt.show()


In [ ]:
X_train, X_test, y_train, y_test = cv.train_test_split(X_iris, y_iris, test_size=0.4, random_state=42)

fig = plt.figure(figsize=(8, 8))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm.Set1, s=40)
plt.show()

In [ ]:
model = LogisticRegression(eps=0.01)
model.fit(X_train, y_train)
probas = model.predict_proba(X_test)
classes = model.classes

labels = []
for pr in probas:
    labels.append(classes[np.argmax(pr)])
print y_test
print np.array(labels)

Реализуем метрику качества, используемую в соревновании: площадь под ROC кривой

In [ ]:
def auroc(y_prob, y_true, label):
    prob = y_prob[:, label-1]
    fpr, tpr, thresholds = sm.roc_curve(y_true, prob, pos_label=label)
    print fpr
    print tpr
    area = np.trapz(tpr, fpr)
    print area
    return area

Разделим выборку с помощью методики кросс-валидации для того, чтобы настроить параметр регуляризации $C$

In [ ]:
C = [0.0, 0.01, 0.1, 1, 10, 100, 1000, 10000]

def select_reg_parameter(C, X, Y):
    auc = []
    X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=0.4)
    for c in C:
        model = LogisticRegression(c, eps=0.001)
        model.fit(X_train, y_train)
        probas = model.predict_proba(X_test)
        au0 = auroc(probas, y_test, 1)
        au1 = auroc(probas, y_test, 2)
        au2 = auroc(probas, y_test, 3)
        print au0, au1, au2
        mean_auc = (au0 + au1 + au2) / 3
        auc.append(mean_auc)
        print "C = " + str(c) + " mean_auc = " + str(mean_auc)
    
    print auc
    return C.index(max(auc))

index = select_reg_parameter(C, X1, Y)
print index


Выбираем наилучшее значение $C$, и классифицируем неизвестных пользователей и строим ROC-кривую

In [ ]:
def predict(X, Y, test_size, C):
    X_train, X_test, y_train, y_test = cv.train_test_split(X, Y, test_size=test_size)
    model = LogisticRegression(C=C)
    model.fit(X_train, y_train)
    probas = model.predict_proba(X_test)
    return y_test, probas

def roc(Y_test, Y_prob, y_prob_ind, pos_label):
    prob = Y_prob[:, y_prob_ind]
    fpr, tpr, thresholds = sm.roc_curve(Y_test, prob, pos_label=pos_label)
    roc_auc = np.trapz(tpr, fpr)
    return tpr, fpr, roc_auc

def plot_roc_curve(tpr, fpr, roc_auc):
    """Plot ROC curve"""
    # Your code here
    fig = plt.figure(figsize=(7, 7))
    ax = fig.gca()
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    plt.text(1, 1, "AUC = " + str(roc_auc))
    plt.plot(fpr, tpr)
    plt.show()
    return

Y_test, Y_prob = predict(X1, Y, 0.3, 1)

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 0, 1)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 1, 2)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

tpr, fpr, roc_auc = roc(Y_test, Y_prob, 2, 3)
print "Area under the ROC curve : %f" % roc_auc
plot_roc_curve(tpr, fpr, roc_auc)

С помощью полученной модели предсказываем категории для неизвестных пользователей из соревнования и загружаем на kaggle в нужном формате.